In [32]:
import pandas as pd
import numpy as np

#Updated as of 8/25 9:43AM

#Data from baseball reference
pitchers = pd.read_csv('/Users/chrisgalgano/desktop/FantasyBBValues/2023_Standard_Pitching.csv')
starters = pd.read_csv('/Users/chrisgalgano/desktop/FantasyBBValues/2023_Starters_Pitching.csv')
relievers = pd.read_csv('/Users/chrisgalgano/desktop/FantasyBBValues/2023_Relievers_Pitching.csv')

pitchers = pitchers.drop_duplicates(subset="Name-additional")
starters = starters.drop_duplicates(subset="Name-additional")
relievers = relievers.drop_duplicates(subset="Name-additional")


pitchers = pd.merge(pitchers,starters[['QS', 'Name-additional']],how = 'left', on = 'Name-additional')
pitchers = pd.merge(pitchers, relievers[['Hold', 'Name-additional']],how = 'left',on = 'Name-additional')

#Determine whether pitcher should be considered as SP or RP
pp = []
for item in pitchers['GS']:
    temp = ''
    if item > 3:
        temp = 'SP'
    else:
        temp = 'RP'
    pp.append(temp)

pitchers['Pos'] = pp

pitchers = pitchers.fillna(0)

start = pitchers.loc[pitchers['Pos'] == 'SP']
start = start.drop((start[start['IP'] < 40] ).index)
start = start.drop((start[start['QS'] < 3 ] ).index)


rel = pitchers.loc[pitchers['Pos'] == 'RP']
rel = rel.drop((rel[rel['IP'] < 30] ).index)
rel = rel.drop(rel[(rel['Hold'] < 5) & (rel['SV'] <5 )].index)

pitchers = pd.concat([start, rel])
pitchers = pitchers.drop_duplicates()
pitchers = pitchers.drop((pitchers[pitchers['Lg'] == 'MLB' ] ).index)


pitchers.to_csv('pitchers.csv', index= False)
pitchers

,Rk,Name,Age,Tm,Lg,W,L,W-L%,ERA,G,...,WHIP,H9,HR9,BB9,SO9,SO/W,Name-additional,QS,Hold,Pos
1,2,Andrew Abbott*,24,CIN,NL,8,3,0.727,3.16,15,...,1.172,7.3,1.3,3.3,9.8,3.00,abbotan01,8.0,0.0,SP
14,15,Sandy Alcantara,27,MIA,NL,6,11,0.353,4.16,26,...,1.193,8.4,1.1,2.3,7.6,3.27,alcansa01,12.0,0.0,SP
18,19,Logan Allen*,24,CLE,AL,6,6,0.500,3.31,19,...,1.335,8.5,1.0,3.5,8.7,2.50,allenlo02,7.0,0.0,SP
25,28,Tyler Anderson*,33,LAA,AL,5,5,0.500,5.35,23,...,1.530,9.8,1.0,4.0,7.6,1.92,anderty01,6.0,0.0,SP
28,31,Graham Ashcraft,25,CIN,NL,7,8,0.467,4.84,24,...,1.381,9.1,1.4,3.4,6.7,2.00,ashcrgr01,14.0,0.0,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,937,Devin Williams,28,MIL,NL,7,3,0.700,1.49,51,...,0.952,4.3,0.6,4.3,13.4,3.13,willide03,0.0,0.0,RP
784,944,Steven Wilson,28,SDP,NL,1,1,0.500,3.25,51,...,1.101,5.8,1.2,4.1,9.7,2.38,wilsost02,0.0,22.0,RP
786,946,Josh Winckowski,25,BOS,AL,3,1,0.750,3.06,47,...,1.433,10.2,1.1,2.7,8.4,3.15,winckjo01,0.0,15.0,RP
803,965,Kirby Yates,36,ATL,NL,7,1,0.875,2.90,50,...,1.188,5.1,1.3,5.6,12.0,2.13,yateski01,0.0,7.0,RP


In [33]:
stats = pitchers[['Name', 'Pos', 'IP', 'SO', 'QS', 'W', 'SV', 'Hold', 'ERA', 'WHIP', 'ER', 'H', 'BB']]
#Calculating league average ERA and WHIP
lg_ERAavg = (9 *np.sum(stats['ER'])/np.sum(stats['IP']))
lg_WHIPavg = (np.sum(stats['H']) + np.sum(stats['BB']))/ np.sum(stats['IP'])


In [34]:
#Calculating Z-score for each category
stats['Z_SO'] = round((stats['SO']-np.mean(stats['SO']))/np.std(stats['SO']),3)
stats['Z_QS'] = round((stats['QS']-np.mean(stats['QS']))/np.std(stats['QS']),3)
stats['Z_W'] = round((stats['W']-np.mean(stats['W']))/np.std(stats['W']),3)
stats['Z_SV'] = round((stats['SV']-np.mean(stats['SV']))/np.std(stats['SV']),3)
stats['Z_HOLD'] = round((stats['Hold']-np.mean(stats['Hold']))/np.std(stats['Hold']),3)

#Calculating ERA and WHIP over average
stats['EraOA'] = round(stats['ERA'] - (9*stats['IP']*lg_ERAavg),1)
stats['Z_ERA'] = round(((stats['EraOA']-np.mean(stats['EraOA']))/(np.std(stats['EraOA']))/4),3)
stats['WhipOA'] = round((stats['H']+stats['BB']) - (stats['IP'] * lg_WHIPavg),1)
stats['Z_WHIP'] = round(((stats['WhipOA']-np.mean(stats['WhipOA']))/(np.std(stats['WhipOA']))/4),3)

/var/folders/xb/k5jn_45j51v9dkktpk3b41ch0000gn/T/ipykernel_60137/2061098377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats['Z_SO'] = round((stats['SO']-np.mean(stats['SO']))/np.std(stats['SO']),3)
/var/folders/xb/k5jn_45j51v9dkktpk3b41ch0000gn/T/ipykernel_60137/2061098377.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats['Z_QS'] = round((stats['QS']-np.mean(stats['QS']))/np.std(stats['QS']),3)
/var/folders/xb/k5jn_45j51v9dkktpk3b41ch0000gn/T/ipykernel_60137/2061098377.py:4: SettingWithCop

In [35]:
zdata = stats[['Name','Pos','IP','SO','QS','W','SV','Hold','ERA', 'WHIP', 'Z_SO',
               'Z_QS','Z_W','Z_SV','Z_HOLD','Z_ERA', 'Z_WHIP']]

#Group by positions
pos_z = zdata.groupby('Pos',as_index=False)['Z_SO','Z_QS','Z_W','Z_SV','Z_HOLD','Z_ERA','Z_WHIP'].mean()
pos_z.columns=['Pos','AVGZ_SO','AVGZ_QS','AVGZ_W','AVGZ_SV','AVGZ_HOLD','AVGZ_ERA','AVGZ_WHIP']
replace_z = zdata.groupby('Pos',as_index=False)['Z_SO','Z_QS','Z_W','Z_SV','Z_HOLD','Z_ERA','Z_WHIP'].min()
replace_z.columns=['Pos','RepZ_SO','RepZ_QS','RepZ_W','RepZ_SV','RepZ_HOLD','RepZ_ERA','RepZ_WHIP']

z_changes = pd.merge(pos_z,replace_z,on='Pos')
z_changes=z_changes.round(decimals=3)

zdata = pd.merge(zdata,z_changes,how='left',on='Pos')
zdata

/var/folders/xb/k5jn_45j51v9dkktpk3b41ch0000gn/T/ipykernel_60137/333828384.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pos_z = zdata.groupby('Pos',as_index=False)['Z_SO','Z_QS','Z_W','Z_SV','Z_HOLD','Z_ERA','Z_WHIP'].mean()
/var/folders/xb/k5jn_45j51v9dkktpk3b41ch0000gn/T/ipykernel_60137/333828384.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  replace_z = zdata.groupby('Pos',as_index=False)['Z_SO','Z_QS','Z_W','Z_SV','Z_HOLD','Z_ERA','Z_WHIP'].min()


,Name,Pos,IP,SO,QS,W,SV,Hold,ERA,WHIP,...,AVGZ_HOLD,AVGZ_ERA,AVGZ_WHIP,RepZ_SO,RepZ_QS,RepZ_W,RepZ_SV,RepZ_HOLD,RepZ_ERA,RepZ_WHIP
0,Andrew Abbott*,SP,85.1,93,8.0,8,0,0.0,3.16,1.172,...,-0.764,-0.196,0.025,-1.266,-0.251,-1.581,-0.424,-0.772,-0.578,-0.601
1,Sandy Alcantara,SP,171.0,144,12.0,6,0,0.0,4.16,1.193,...,-0.764,-0.196,0.025,-1.266,-0.251,-1.581,-0.424,-0.772,-0.578,-0.601
2,Logan Allen*,SP,103.1,100,7.0,6,0,0.0,3.31,1.335,...,-0.764,-0.196,0.025,-1.266,-0.251,-1.581,-0.424,-0.772,-0.578,-0.601
3,Tyler Anderson*,SP,117.2,100,6.0,5,0,0.0,5.35,1.530,...,-0.764,-0.196,0.025,-1.266,-0.251,-1.581,-0.424,-0.772,-0.578,-0.601
4,Graham Ashcraft,SP,134.0,100,14.0,7,0,0.0,4.84,1.381,...,-0.764,-0.196,0.025,-1.266,-0.251,-1.581,-0.424,-0.772,-0.578,-0.601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,Devin Williams,RP,48.1,72,0.0,7,30,0.0,1.49,0.952,...,0.769,0.198,-0.025,-1.418,-0.819,-1.581,-0.424,-0.772,0.024,-0.491
267,Steven Wilson,RP,52.2,57,0.0,1,0,22.0,3.25,1.101,...,0.769,0.198,-0.025,-1.418,-0.819,-1.581,-0.424,-0.772,0.024,-0.491
268,Josh Winckowski,RP,67.2,63,0.0,3,3,15.0,3.06,1.433,...,0.769,0.198,-0.025,-1.418,-0.819,-1.581,-0.424,-0.772,0.024,-0.491
269,Kirby Yates,RP,49.2,66,0.0,7,2,7.0,2.90,1.188,...,0.769,0.198,-0.025,-1.418,-0.819,-1.581,-0.424,-0.772,0.024,-0.491


In [36]:
#Subtract z-score by replacement level player
zdata['Z_SO'] = zdata['Z_SO'] - zdata['RepZ_SO']
zdata['Z_QS'] = zdata['Z_QS'] - zdata['RepZ_QS']
zdata['Z_W'] = zdata['Z_W'] - zdata['RepZ_W']
zdata['Z_SV'] = zdata['Z_SV'] - zdata['RepZ_SV']
zdata['Z_HOLD'] = zdata['Z_HOLD'] - zdata['RepZ_HOLD']
zdata['Z_ERA'] = zdata['Z_ERA'] - zdata['RepZ_ERA']
zdata['Z_WHIP'] = zdata['Z_WHIP'] - zdata['RepZ_WHIP']

#Add up Z-scores to determine overall score
zdata['Score'] = (zdata['Z_SO'] + zdata['Z_QS'] + zdata['Z_W'] + zdata['Z_SV'] + 
                  zdata['Z_HOLD'] - zdata['Z_ERA'] - zdata['Z_WHIP'])/7
zdata = zdata.sort_values(by='Score',ascending=False)


totaldollars = (272*12*.4)
print('{}$ to spend'.format(totaldollars))

perplayer = round(totaldollars/len(zdata),1)
print('Average ${}/player'.format(perplayer))

zdata['ZAR'] = zdata['Score']/np.mean(zdata['Score'])
zdata['$'] = round(zdata['ZAR']*perplayer,2)

printdf = zdata[['Name','Pos','IP','SO','QS','W','SV','Hold','ERA','WHIP','Score','$']] 




printdf.to_csv('PitcherPlayerValues.csv', index= False)

1305.6000000000001$ to spend
Average $4.8/player
